In [1]:
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt

from tqdm import tqdm

In [2]:
def get_accelleration(pos, mass, G, softening):
    """ Calculate the acceleration on each particle due to Newton's Law 
        * pos is an N x 3 matrix of positions
        * mass is an N x 1 vector of masses
        * G is Newton's Gravitational constant
        * softening is the softening length
        * a is N x 3 matrix of accelerations
    """
    # positions r = [x,y,z] for all particles
    x = pos[:,0:1]
    y = pos[:,1:2]
    z = pos[:,2:3]

    # matrix that stores all pairwise particle separations: r_j - r_i
    dx = x.T - x
    dy = y.T - y
    dz = z.T - z
    
    # matrix that stores 1/r^3 for all particle pairwise particle separations 
    inv_r3 = (dx**2 + dy**2 + dz**2 + softening**2)**(-1.5)
    #inv_r3[inv_r3>0] = inv_r3[inv_r3>0]**(-1.5)

    ax = G * (dx * inv_r3) @ mass
    ay = G * (dy * inv_r3) @ mass
    az = G * (dz * inv_r3) @ mass

    # pack together the acceleration components
    a = np.hstack((ax,ay,az))

    return a


In [3]:
%%time
N         = 100    # Number of particles
softening = 0.1    # softening length
G         = 1.0    # Newton's Gravitational Constant

mass = 20.0*np.ones((N,1))/N  # total mass of particles is 20
pos  = np.random.randn(N,3) 

#mass = np.array([[10.], [10.]])
#pos = np.array([[0.5, 0.5, 0.0], [-0.5, -0.5, 0.0]])

a = get_accelleration(pos, mass, G, softening)
#a

CPU times: user 4.38 ms, sys: 3.17 ms, total: 7.54 ms
Wall time: 2.61 ms


In [4]:
%%time
N         = 10000    # Number of particles
mass = 20.0*cp.ones((N,1))/N  # total mass of particles is 20
pos  = cp.random.randn(N,3) 
G = cp.array(1.0)
softening = cp.array(0.1)

#mass = cp.array([[10.], [10.]])
#pos = cp.array([[0.5, 0.5, 0.0], [-0.5, -0.5, 0.0]])
#a = get_accelleration(pos, mass, G, softening)
a = get_accelleration(pos, mass, G, softening)
#a

CPU times: user 1.23 s, sys: 358 ms, total: 1.59 s
Wall time: 1.96 s


# N-body simulation

In [5]:
# Simulation parameters
N         = 1000    # Number of particles
t         = 0      # current time of the simulation
tEnd      = 100.0   # time at which simulation ends
dt        = 0.01   # timestep
softening = 0.1    # softening length
G         = 1.0    # Newton's Gravitational Constant

#gpu = False

# Generate Initial Conditions
np.random.seed(2024)            # set the random number generator seed

#mass = np.array([[1000.0],[10.0]]) 
#pos = np.array([[1e-10, 1e-10, 1e-10], [-1.0, -1.0, 0.0]])
#vel = np.array([[0.0, 0.0, 0.0], [-3.0, 3.0, 0.0]])

#mass = 20.0*np.ones((N,1))/N  # total mass of particles is 20
#pos = np.array([[0.5, 0.5, 0.0], [-0.5, -0.5, 0.0]])
#vel = np.array([[0.0, 1.5, 0.0], [0.0, -1.5, 0.0]])
#pos  = np.random.randn(N,3)   # randomly selected positions and velocities
#vel  = np.random.randn(N,3)

mass = 20.0*cp.ones((N,1))/N  # total mass of particles is 20
vel  = cp.random.randn(N,3)
pos  = cp.random.randn(N,3) 
G = cp.array(G)
softening = cp.array(softening)
dt = cp.array(dt)

# Convert to Center-of-Mass frame
#vel -= np.mean(mass * vel,0) / np.mean(mass)

# calculate initial gravitational accelerations
acc = get_accelleration(pos, mass, G, softening)

# number of timesteps
Nt = int(np.ceil(tEnd/dt))

# save energies, particle orbits for plotting trails
pos_save = cp.zeros((N,3,Nt+1))
pos_save[:,:,0] = pos
#pos_save[:,:,0] = pos.get()

t_all = cp.arange(Nt+1)*dt

# Simulation Main Loop
for i in tqdm(range(Nt)):
    # (1/2) kick
    vel += acc * dt/2.0

    # drift
    pos += vel * dt

    # update accelerations
    acc = get_accelleration(pos, mass, G, softening )

    # (1/2) kick
    vel += acc * dt/2.0

    # update time
    t += dt

    # save energies, positions for plotting trail
    #pos_save[:,:,i+1] = pos.get()
    pos_save[:,:,i+1] = pos
       
pos_save = pos_save.get()
mass = mass.get()

100%|████████████████████████████████████| 10000/10000 [01:18<00:00, 127.67it/s]


## Plot

In [6]:
for i in tqdm(range(Nt)[slice(None,None,10)]):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(projection='3d')
    
    ax.grid(True, color='grey', alpha=0.2)  # Grid lines with transparency
    ax.set_facecolor('white')
    
    x, y, z = pos_save[...,i].T
    ax.scatter(x, y, z, marker='o', s=10, color='tab:blue')
    if(i < 25):
        ax.scatter(pos_save[:,0,:i].T, pos_save[:,1,:i].T, pos_save[:,2,:i].T, marker='o', s=1, color='tab:orange', alpha=0.2)
    else:
        ax.scatter(pos_save[:,0,i-25:i].T, pos_save[:,1,i-25:i].T, pos_save[:,2,i-25:i].T, marker='o', s=1, color='tab:orange', alpha=0.2)
    
    pos_cm = np.mean(pos_save[...,i]*mass, axis=0)/np.mean(mass)
    ax.set_xlim(pos_cm[0]-2.0, pos_cm[0]+2.0), ax.set_ylim(pos_cm[1]-2.0, pos_cm[1]+2.0), ax.set_zlim(pos_cm[2]-2.0, pos_cm[2]+2.0)
    
    ax.set_xlabel('x'), ax.set_ylabel('y'), ax.set_zlabel('z')
    
    plt.savefig('nbody_snapshot/snapshot_%d.png' %i, bbox_inches='tight')
    plt.clf(), plt.close()

100%|███████████████████████████████████████| 1000/1000 [16:54<00:00,  1.01s/it]


## Create GIF

In [37]:
from moviepy.editor import ImageSequenceClip

def CreateMovie(filename, array, fps=5, scale=1., fmt='avi'):
    ''' Create and save a gif or video from array of images.
        Parameters:
            * filename (string): name of the saved video
            * array (list or string): array of images name already in order, if string it supposed to be the first part of the images name (before iteration integer)
            * fps = 5 (integer): frame per seconds (limit human eye ~ 15)
            * scale = 1. (float): ratio factor to scale image hight and width
            * fmt (string): file extention of the gif/video (e.g: 'gif', 'mp4' or 'avi')
        Return:
            * moviepy clip object
    '''
    if(isinstance(array, str)):
        array = np.array(sorted(glob(array+'*.png'), key=os.path.getmtime))
    else:
        pass
    filename += '.'+fmt
    clip = ImageSequenceClip(list(array), fps=fps).resize(scale)
    if(fmt == 'gif'):
        clip.write_gif(filename, fps=fps)
    elif(fmt == 'mp4'):
        clip.write_videofile(filename, fps=fps, codec='mpeg4')
    elif(fmt == 'avi'):
        clip.write_videofile(filename, fps=fps, codec='png')
    else:
        print('Error! Wrong File extension.')
        sys.exit()

    # print the size of the movie
    command = os.popen('du -sh %s' % filename)
    print(command.read())
    return clip


ModuleNotFoundError: No module named 'moviepy'

In [ ]:
path_out = './nbody_snapshot/'
img_arr = ['%ssnapshot_%d.png' %(path_out, i) for i in range(len(glob(path_out+'*png')))]

#CreateMovie(filename='movie_HI_IGM_EoR', array=img_arr, fps=15, fmt='avi')
CreateMovie(filename='movie_HI_IGM_EoR', array=img_arr, fps=15, fmt='mp4')